In [3]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
import random

In [4]:
# specify directory and CSV file paths
system = "jupyter"
data = "dag"

if system == "linux":
    data_dir = "/fp/homes01/u01/ec-gerald/My Projects/ec192/data/endo-radiographs/clips"
elif system == "win":
    data_dir = r"\\aspasia.ad.fp.educloud.no\ec192\data\endo-radiographs\clips"
elif system == "jupyter":
    if data == "dag":
        data_dir = "/fp/projects01/ec192/data/endo-radiographs/dag/clips"
    elif data == "deniz":
        data_dir = "/fp/projects01/ec192/data/endo-radiographs/deniz/clips"
csv_file = 'codefile.csv'

In [7]:
def load_data(data_dir):
    df = pd.read_csv(os.path.join(data_dir, "codefile.csv"))
    images = []
    labels = []

    for _, row in df.iterrows():
        img = Image.open(os.path.join(data_dir, row['filename']))
        img = img.resize((256, 256))
        img = np.array(img)
        img = img.reshape((256, 256, 1))  # Reshape to add the channel dimension
        images.append(img)

        # Subtract 1 from the label to make them 0-based (i.e., from 1-5 to 0-4)
        labels.append(row['prediction'] - 1)

    return np.array(images), np.array(labels)

X, y = load_data(data_dir)

# One-hot encode the labels
y = to_categorical(y, num_classes=5)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(256, 256, 1)))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    #model.add(Dense(64, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))  # output layer with 5 nodes

    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = create_model()

def add_noise(img):
    '''Add random noise to an image'''
    VARIABILITY = 50
    deviation = VARIABILITY*random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img

datagen = ImageDataGenerator(
    rotation_range=10,  # Rotate images up to 10 degrees
    zoom_range=0.1,  # Zoom in/out up to 10%
    horizontal_flip=True,  # Allow horizontal flipping
    fill_mode='nearest',  # Fill in newly created pixels
    preprocessing_function=add_noise  # Add noise
)

datagen.fit(X_train)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) // 32,
                    validation_data=(X_test, y_test),
                    epochs=50)

plt.figure(figsize=[6,4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy Curves', fontsize=12)
plt.show()

plt.figure(figsize=[6,4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
plt.plot(history.history['val_loss'], 'blue', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)

Epoch 1/50


2023-08-09 14:43:03.546753: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


57/57 [==============================] - 24s 405ms/step - loss: 11.7059 - accuracy: 0.3836 - val_loss: 1.6068 - val_accuracy: 0.4341
Epoch 2/50
57/57 [==============================] - 22s 387ms/step - loss: 1.6053 - accuracy: 0.4155 - val_loss: 1.6036 - val_accuracy: 0.4341
Epoch 3/50
57/57 [==============================] - 23s 402ms/step - loss: 1.6021 - accuracy: 0.4177 - val_loss: 1.6003 - val_accuracy: 0.4341
Epoch 4/50
57/57 [==============================] - 22s 378ms/step - loss: 1.6002 - accuracy: 0.4177 - val_loss: 1.5971 - val_accuracy: 0.4341
Epoch 5/50
57/57 [==============================] - 23s 400ms/step - loss: 1.8488 - accuracy: 0.4172 - val_loss: 1.5940 - val_accuracy: 0.4341
Epoch 6/50
57/57 [==============================] - 22s 381ms/step - loss: 1.5927 - accuracy: 0.4172 - val_loss: 1.5908 - val_accuracy: 0.4341
Epoch 7/50
57/57 [==============================] - 22s 387ms/step - loss: 1.5895 - accuracy: 0.4155 - val_loss: 1.5877 - val_accuracy: 0.4341
Epoch 8/5